In [74]:
# 03_train_lstm.ipynb  —  Block 3 (mit WFCV-Bestkonfig-Übernahme)

import os, sys, json, time, glob
from pathlib import Path
import numpy as np
import pandas as pd

ROOT = os.path.abspath("..")
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

# === 0) Grund-Config laden ====================================================
with open(os.path.join(ROOT, "config.json"), "r") as f:
    C = json.load(f)

TICKER   = C["ticker"]; START = C["start"]; END = C["end"]; INTERVAL = C["interval"]
HORIZON  = int(C["horizon"])
LOOKBACK = int(C["lookback"])                  # kann von best_config.json überschrieben werden
BATCH    = int(C["batch"]); EPOCHS = int(C["epochs"])
SEED     = int(C.get("seed", 42))
FEATURESET = C.get("featureset", "v2")
EPS_MODE   = C.get("epsilon_mode", "abs")
EPSILON    = float(C.get("epsilon", 0.0005))

RESULTS_DIR = Path(C.get("results_dir", "../results"))
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
RUN_DIR   = RESULTS_DIR / time.strftime("%Y-%m-%d_%H-%M-%S_lstm")
RUN_DIR.mkdir(parents=True, exist_ok=True)
print("RUN_DIR:", RUN_DIR)

# Train-CSV wie in Block 2 benannt (inkl. eps_tag)
eps_tag   = f"{EPS_MODE}{str(EPSILON).replace('.','p')}"
TRAIN_CSV = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}_{eps_tag}.csv"

RUN_DIR: ..\results\2025-10-19_16-48-09_lstm


In [75]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import (
    classification_report, confusion_matrix,
    balanced_accuracy_score, matthews_corrcoef, average_precision_score,
    roc_auc_score
)

In [76]:
# === 1) WFCV-best_config.json (neuester Run) laden ============================
def _latest_best_config(results_dir="../results"):
    """Finde den neuesten Ordner, der auf *_wfcv endet, und lese best_config.json."""
    pattern = os.path.join(results_dir, "*_wfcv", "best_config.json")
    cands = glob.glob(pattern)
    if not cands:
        return None, None
    cands = sorted(cands, key=os.path.getmtime)  # nach Modifikationszeit
    best_path = cands[-1]
    with open(best_path, "r") as f:
        best_cfg = json.load(f)
    return best_cfg, best_path

BEST_CFG, BEST_CFG_PATH = _latest_best_config(RESULTS_DIR)
if BEST_CFG is None:
    print("[INFO] Keine best_config.json gefunden — nutze Fallback (Config.json-Defaults).")
    # Fallback: entspricht deiner bisherigen Default-Architektur
    BEST_CFG = {
        "features_used": "all",
        "lookback": LOOKBACK,
        "cell": "GRU",
        "width1": 32,
        "width2": 16,
        "dropout": 0.10,
        "lr": 5e-4
    }
else:
    print("Gefunden best_config.json:", BEST_CFG_PATH)

# HP aus best_config übernehmen (LOOKBACK ggf. überschreiben)
CELL    = str(BEST_CFG.get("cell", "GRU")).upper()
WIDTH1  = int(BEST_CFG.get("width1", 32))
WIDTH2  = int(BEST_CFG.get("width2", 16))
DROPOUT = float(BEST_CFG.get("dropout", 0.10))
LR      = float(BEST_CFG.get("lr", 5e-4))
LB_FROM_BEST = int(BEST_CFG.get("lookback", LOOKBACK))
USE_LOOKBACK = LB_FROM_BEST if LB_FROM_BEST > 0 else LOOKBACK

FEATURES_USED_TAG = str(BEST_CFG.get("features_used", "all"))

print(f"[Block3 Setup] cell={CELL} width={WIDTH1}/{WIDTH2} dp={DROPOUT} lr={LR} lookback={USE_LOOKBACK} "
      f"| features_used={FEATURES_USED_TAG}")

Gefunden best_config.json: ..\results\2025-10-19_16-45-57_wfcv\best_config.json
[Block3 Setup] cell=GRU width=32/16 dp=0.1 lr=0.0005 lookback=60 | features_used=mom_only


In [77]:
# === 2) Daten & Features einlesen (robust zur H/ε-Diskrepanz) ===============
import yaml, glob, os

yaml_path = f"../data/features_{FEATURESET}.yml"
meta = {}
if os.path.exists(yaml_path):
    with open(yaml_path, "r") as f:
        meta = yaml.safe_load(f) or {}

def _resolve_train_csv():
    # 1) Versuch: exakte Datei gemäß aktueller config.json
    eps_tag_cfg = f"{EPS_MODE}{str(EPSILON).replace('.','p')}"
    exact = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}_{eps_tag_cfg}.csv"
    if os.path.exists(exact):
        return exact

    # 2) Versuch: nimm Label-Definition aus YAML (Block 2 ist Quelle der Wahrheit)
    lab = (meta or {}).get("label", {})
    h_yaml   = int(lab.get("horizon", HORIZON))
    mode_yaml= str(lab.get("mode", EPS_MODE))
    eps_yaml = float(lab.get("epsilon", EPSILON))
    eps_tag_yaml = f"{mode_yaml}{str(eps_yaml).replace('.','p')}"
    by_yaml = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{h_yaml}_{eps_tag_yaml}.csv"
    if os.path.exists(by_yaml):
        return by_yaml

    # 3) Versuch: gleiche H/ε (aus YAML), aber beliebiges h via Glob
    pat_same_eps = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h*_{eps_tag_yaml}.csv"
    cands = sorted(glob.glob(pat_same_eps), key=os.path.getmtime)
    if cands:
        return cands[-1]

    # 4) Letzte Instanz: irgendein *_cls_h*.csv für diesen Ticker/Zeitraum
    pat_any = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h*.csv"
    cands = sorted(glob.glob(pat_any), key=os.path.getmtime)
    if cands:
        return cands[-1]

    raise FileNotFoundError(
        "Kein TRAIN_CSV gefunden.\n"
        f"Versucht:\n- {exact}\n- {by_yaml}\n- {pat_same_eps}\n- {pat_any}\n"
        "Hinweis: Block 2 mit identischen HORIZON/epsilon erneut laufen lassen."
    )

TRAIN_CSV = _resolve_train_csv()
print("Loaded TRAIN_CSV:", TRAIN_CSV)

# --- Label-Parameter (H, mode, epsilon) aus Daten erzwingen -------------------
import re

def _infer_label_from(meta_dict, train_csv_path, fallback_h_from_root):
    h, mode, eps = None, None, None

    # 1) YAML als Quelle der Wahrheit (Block 2 schreibt das rein)
    if meta_dict and isinstance(meta_dict, dict):
        L = (meta_dict.get("label") or {})
        if "horizon" in L: h = int(L["horizon"])
        if "mode"    in L: mode = str(L["mode"])
        if "epsilon" in L:
            try:
                eps = float(L["epsilon"])
            except Exception:
                pass

    # 2) Robust: aus Dateinamen …_cls_h{H}_{mode}{eps_tag}.csv parsen
    m = re.search(r"_cls_h(\d+)_([a-z]+)([0-9p]+)\.csv$", str(train_csv_path))
    if m:
        if h    is None: h    = int(m.group(1))
        if mode is None: mode = m.group(2)
        if eps  is None:
            eps_tag = m.group(3)          # z.B. 0p0005
            eps = float(eps_tag.replace("p", "."))

    # 3) Fallback: notfalls Root-H (sollte eigentlich nie passieren)
    if h is None:
        h = int(fallback_h_from_root)

    return h, mode, eps

# erzwingen, dass HORIZON/EPS aus Daten kommen
H_DATA, MODE_DATA, EPS_DATA = _infer_label_from(meta, TRAIN_CSV, HORIZON)

# Diese Werte ab hier im Run *verbindlich* nutzen
HORIZON  = int(H_DATA)
if MODE_DATA is not None:   EPS_MODE = str(MODE_DATA)
if EPS_DATA  is not None:   EPSILON  = float(EPS_DATA)

# für Transparenz
print(f"[Label] using horizon={HORIZON} | mode={EPS_MODE} | epsilon={EPSILON}")


df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=True).sort_index()

# Features aus YAML ziehen und mit df abgleichen
ALL_FEATURES = [c for c in (meta.get("features", []) if meta else []) if c in df.columns]
if not ALL_FEATURES:
    # Fallback: alles außer OHLCV/target
    OHLCV = {"open","high","low","close","volume"}
    ALL_FEATURES = [c for c in df.columns if c not in (OHLCV | {"target"})]
assert len(ALL_FEATURES) > 0, f"Keine nutzbaren Features in {yaml_path} oder df gefunden."

# Feature-Subset konsistent zu best_config.json
if FEATURES_USED_TAG == "mom_only":
    FEATURES = [c for c in ALL_FEATURES
                if ("logret" in c) or ("macd" in c) or (c in {"sma_diff","rsi_14","bb_pos"})]
    assert len(FEATURES) > 0, "Feature-Subset 'mom_only' ist leer — bitte prüfen."
else:
    FEATURES = ALL_FEATURES

TARGET = "target"
X = df[FEATURES].copy()
y = df[TARGET].astype(int).copy()
print("FEATURES (final):", FEATURES)


Loaded TRAIN_CSV: ../data/AAPL_1d_2012-01-01_2025-09-01_cls_h5_abs0p0005.csv
[Label] using horizon=5 | mode=abs | epsilon=0.0005
FEATURES (final): ['logret_1d', 'logret_3d', 'logret_5d', 'bb_pos', 'rsi_14', 'macd', 'macd_sig', 'macd_diff', 'sma_diff']


In [78]:
# === 3) Chronologische Splits (70/15/15) =====================================
from sklearn.preprocessing import StandardScaler

n = len(df)
n_train = int(n * 0.70)
n_val   = int(n * 0.15)
train_idx = slice(0, n_train)
val_idx   = slice(n_train, n_train + n_val)
test_idx  = slice(n_train + n_val, n)

X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
X_val,   y_val   = X.iloc[val_idx],   y.iloc[val_idx]
X_test,  y_test  = X.iloc[test_idx],  y.iloc[test_idx]
print(f"Split sizes → train {len(X_train)}, val {len(X_val)}, test {len(X_test)}")

Split sizes → train 2381, val 510, test 511


In [79]:
print("X_train shape/check:", X_train.shape, " | cols:", list(X_train.columns))


X_train shape/check: (2381, 9)  | cols: ['logret_1d', 'logret_3d', 'logret_5d', 'bb_pos', 'rsi_14', 'macd', 'macd_sig', 'macd_diff', 'sma_diff']


In [80]:
# === 4) Scaler nur auf TRAIN fitten ==========================================
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_s = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=FEATURES)
X_val_s   = pd.DataFrame(scaler.transform(X_val),       index=X_val.index,   columns=FEATURES)
X_test_s  = pd.DataFrame(scaler.transform(X_test),      index=X_test.index,  columns=FEATURES)

import joblib
joblib.dump(scaler, RUN_DIR / "scaler.joblib")

['..\\results\\2025-10-19_16-48-09_lstm\\scaler.joblib']

In [81]:
# Drift-Report (lassen)
def drift_summary(Xa: pd.DataFrame, Xb: pd.DataFrame):
    out = []
    for c in Xa.columns:
        m1, s1 = Xa[c].mean(), Xa[c].std(ddof=1)
        m2, s2 = Xb[c].mean(), Xb[c].std(ddof=1)
        ratio_std = float((s2 + 1e-9) / (s1 + 1e-9))
        diff_mean = float(m2 - m1)
        out.append({"feature": c, "mean_diff": diff_mean, "std_ratio": ratio_std})
    return pd.DataFrame(out).sort_values("std_ratio", ascending=False)

drift_df = drift_summary(X_train_s, X_test_s)
drift_df.to_csv(RUN_DIR / "drift_train_vs_test.csv", index=False)

In [82]:
# === 5) Windowing & tf.data ===================================================
def make_windows(X_df: pd.DataFrame, y_ser: pd.Series, lookback: int):
    X_values = X_df.values.astype(np.float32)
    y_values = y_ser.values.astype(np.int32)
    n = len(X_df)
    xs, ys = [], []
    for i in range(lookback-1, n):
        xs.append(X_values[i - lookback + 1 : i + 1])
        ys.append(y_values[i])
    return np.stack(xs, axis=0), np.array(ys)

Xtr_win, ytr = make_windows(X_train_s, y_train, USE_LOOKBACK)
Xva_win, yva = make_windows(X_val_s,   y_val,   USE_LOOKBACK)
Xte_win, yte = make_windows(X_test_s,  y_test,  USE_LOOKBACK)

import tensorflow as tf
np.random.seed(SEED); tf.random.set_seed(SEED)

def to_ds(X, y, batch, shuffle):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X), seed=SEED, reshuffle_each_iteration=True)
    return ds.batch(batch).prefetch(tf.data.AUTOTUNE)

ds_train = to_ds(Xtr_win, ytr, BATCH, shuffle=True)
ds_val   = to_ds(Xva_win, yva, BATCH, shuffle=False)
ds_test  = to_ds(Xte_win, yte, BATCH, shuffle=False)

In [83]:
# === 6) Diagnose: LogReg (Ranking-Baseline) ==================================
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, matthews_corrcoef

logit = LogisticRegression(max_iter=200)
logit.fit(X_train_s.iloc[USE_LOOKBACK-1:], y_train.iloc[USE_LOOKBACK-1:])
y_proba_lr = logit.predict_proba(X_test_s.iloc[USE_LOOKBACK-1:])[:,1]
print(f"[Diag] LogReg AUROC val/test = "
      f"{roc_auc_score(y_val.iloc[USE_LOOKBACK-1:], logit.predict_proba(X_val_s.iloc[USE_LOOKBACK-1:])[:,1]):.3f}/"
      f"{roc_auc_score(y_test.iloc[USE_LOOKBACK-1:], y_proba_lr):.3f}")

# === 7) Modell nach best_config.json =========================================
from tensorflow.keras import layers, regularizers, optimizers, callbacks, models

rnn_cell = layers.GRU if CELL == "GRU" else layers.LSTM

model = models.Sequential([
    layers.Input(shape=(USE_LOOKBACK, len(FEATURES))),
    rnn_cell(WIDTH1, return_sequences=True, recurrent_dropout=DROPOUT),
    layers.LayerNormalization(),
    rnn_cell(WIDTH2, recurrent_dropout=DROPOUT),
    layers.LayerNormalization(),
    layers.Dense(16, activation="relu", kernel_regularizer=regularizers.l2(1e-5)),
    layers.Dense(1, activation="sigmoid"),
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=LR),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.AUC(name="auprc", curve="PR"),
        tf.keras.metrics.BinaryAccuracy(name="acc"),
        tf.keras.metrics.Precision(name="prec"),
        tf.keras.metrics.Recall(name="rec"),
    ],
)

ckpt_path = RUN_DIR / "best.keras"
cbs = [
    callbacks.ModelCheckpoint(filepath=str(ckpt_path),
                              monitor="val_auprc", mode="max",
                              save_best_only=True, verbose=1),
    callbacks.EarlyStopping(monitor="val_auprc", mode="max",
                            patience=12, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor="val_auprc", mode="max",
                                factor=0.5, patience=6, min_lr=1e-5, verbose=1),
]

[Diag] LogReg AUROC val/test = 0.585/0.446


In [84]:
# === 8) Training ==============================================================
history = model.fit(ds_train, validation_data=ds_val, epochs=EPOCHS,
                    callbacks=cbs, verbose=1)
pd.DataFrame(history.history).to_csv(RUN_DIR / "history.csv", index=False)

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.5354 - auc: 0.4977 - auprc: 0.5914 - loss: 0.7758 - prec: 0.5888 - rec: 0.7098
Epoch 1: val_auprc improved from None to 0.61817, saving model to ..\results\2025-10-19_16-48-09_lstm\best.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - acc: 0.5323 - auc: 0.4977 - auprc: 0.5811 - loss: 0.7447 - prec: 0.5709 - rec: 0.7634 - val_acc: 0.4989 - val_auc: 0.5516 - val_auprc: 0.6182 - val_loss: 0.7149 - val_prec: 0.5171 - val_rec: 0.7605 - learning_rate: 5.0000e-04
Epoch 2/100
33/37 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.5330 - auc: 0.5011 - auprc: 0.5790 - loss: 0.6992 - prec: 0.5610 - rec: 0.8028
Epoch 2: val_auprc improved from 0.61817 to 0.62571, saving model to ..\results\2025-10-19_16-48-09_lstm\best.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.5543 - auc: 0.5204 - auprc: 0.6033 - loss: 0.6881 - prec: 0.5817 - rec: 0.8104 - val_acc: 0.5410 - val_auc: 0.6087 - val_auprc: 0.6257 - val_loss: 0.6815 - val_prec: 0.5447 -

In [85]:
# === 9) Test (roh, ohne Kalibrierung/Final-Threshold) ========================
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, average_precision_score

In [86]:
# Metriken im Keras-Sinne
test_metrics = model.evaluate(ds_test, return_dict=True, verbose=0)
print("Test (keras) metrics:", json.dumps(test_metrics, indent=2))

Test (keras) metrics: {
  "acc": 0.4690265357494354,
  "auc": 0.47294431924819946,
  "auprc": 0.5432189702987671,
  "loss": 0.7261859178543091,
  "prec": 0.49666666984558105,
  "rec": 0.6260504126548767
}


In [87]:
# Diagnose-Threshold: val-MCC innerhalb 0.35–0.65 PosRate (nur Diagnose!)
val_proba = model.predict(ds_val, verbose=0).ravel()
def choose_threshold(y_true, y_prob, bounds=(0.35, 0.65)):
    uniq = np.unique(y_prob); cand = np.r_[0.0, uniq, 1.0]
    best_t, best_s = 0.5, -1
    for t in cand:
        yp = (y_prob >= t).astype(int)
        pr = yp.mean()
        if not (bounds[0] <= pr <= bounds[1]): 
            continue
        s = matthews_corrcoef(y_true, yp)
        if s > best_s: best_s, best_t = s, float(t)
    return best_t, best_s

thr_diag, mcc_val_diag = choose_threshold(yva, val_proba, bounds=(0.35, 0.65))
print(f"[Diag] thr@val(max MCC, bounds 0.35–0.65) = {thr_diag:.3f} | val_MCC={mcc_val_diag:.3f}")

[Diag] thr@val(max MCC, bounds 0.35–0.65) = 0.594 | val_MCC=0.240


In [88]:
# Test-Probas & Diagnose-Report bei thr_diag (NICHT final reporten; Block 4 entscheidet!)
y_proba = model.predict(ds_test, verbose=0).ravel()
y_pred_diag = (y_proba >= thr_diag).astype(int)

print("Proba stats (test): min=", float(y_proba.min()),
      "max=", float(y_proba.max()), "mean=", float(y_proba.mean()))
print("AUROC val/test:",
      round(roc_auc_score(yva, val_proba), 3), "/",
      round(roc_auc_score(yte, y_proba), 3))

print("\n[Diag] Confusion (test, thr=thr_diag):\n", confusion_matrix(yte, y_pred_diag))
print("\n[Diag] Report (test):\n", classification_report(yte, y_pred_diag, digits=3))

# Extra Test-Metriken (diagnostisch)
bal_acc = balanced_accuracy_score(yte, y_pred_diag)
mcc     = matthews_corrcoef(yte, y_pred_diag)
auprc_t = average_precision_score(yte, y_proba)
extra = {"balanced_accuracy": float(bal_acc),
         "mcc": float(mcc),
         "auprc": float(auprc_t)}
with open(RUN_DIR / "extra_test_metrics_diag.json", "w") as f:
    json.dump(extra, f, indent=2)

Proba stats (test): min= 0.3109811544418335 max= 0.7283353805541992 mean= 0.5499657988548279
AUROC val/test: 0.61 / 0.473

[Diag] Confusion (test, thr=thr_diag):
 [[121  93]
 [139  99]]

[Diag] Report (test):
               precision    recall  f1-score   support

           0      0.465     0.565     0.511       214
           1      0.516     0.416     0.460       238

    accuracy                          0.487       452
   macro avg      0.491     0.491     0.486       452
weighted avg      0.492     0.487     0.484       452



In [89]:
# === 10) Artefakte & Umgebungsinfo (ein konsistenter Dump) ===================
env_info = {
    "python": sys.version,
    "tensorflow": tf.__version__,
    "seed": SEED,
    "ticker": TICKER, "start": START, "end": END, "interval": INTERVAL,
    "horizon": HORIZON, "epsilon_mode": EPS_MODE, "epsilon": EPSILON,
    "featureset": FEATURESET, "features_used": FEATURES_USED_TAG,
    "features_final": FEATURES,
    "lookback": USE_LOOKBACK, "batch": BATCH, "epochs": EPOCHS,
    "cell": CELL, "width1": WIDTH1, "width2": WIDTH2,
    "dropout": DROPOUT, "lr": LR,
    "loss": "BCE",
    "train_csv": TRAIN_CSV,
    "features_yaml": yaml_path,
    "best_config_path": BEST_CFG_PATH,
    "best_checkpoint_path": str(ckpt_path)
}
with open(RUN_DIR / "env_info.json", "w") as f:
    json.dump(env_info, f, indent=2)

In [90]:
# kompakter Config-Dump (einmalig!) — keine Doppel-Schreiberei
final_cfg_dump = {
    "ticker": TICKER, "start": START, "end": END, "interval": INTERVAL,
    "horizon": HORIZON, "lookback": USE_LOOKBACK,
    "featureset": FEATURESET, "features": FEATURES,
    "scaler": "StandardScaler", "seed": SEED, "batch": BATCH, "epochs": EPOCHS,
    "cell": CELL, "width1": WIDTH1, "width2": WIDTH2,
    "dropout": DROPOUT, "lr": LR,
    "loss": "BCE",
    "epsilon_mode": EPS_MODE, "epsilon": EPSILON,
    "train_csv": TRAIN_CSV,                         
    "features_yaml": yaml_path,
    "wfcv_best_config_source": BEST_CFG_PATH
}
with open(RUN_DIR / "config.json", "w") as f:
    json.dump(final_cfg_dump, f, indent=2)

In [91]:
# Keras-Export & Rohdaten für Block 4
model.save(RUN_DIR / "model.keras")
np.save(RUN_DIR / "y_test.npy", yte)
np.save(RUN_DIR / "y_proba.npy", y_proba)

print(f"\nArtefakte gespeichert in: {RUN_DIR}")


Artefakte gespeichert in: ..\results\2025-10-19_16-48-09_lstm
